<a href="https://colab.research.google.com/github/kolayn808/textattack_test/blob/main/test_seq2sick.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Установка библиотек

In [2]:
!pip install transformers
!pip install nltk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 28.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 53.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 65.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 31.0 MB/s eta 0:00:00


Импорт необходимых библиотек

In [3]:
import random
import nltk
import gensim.downloader
from nltk.tokenize import word_tokenize
from gensim.models import Word2Vec
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import T5Tokenizer, T5ForConditionalGeneration
from termcolor import colored
import zlib

Загрузка моделей и стоп-слов

In [4]:
# Загрузка модели Word2Vec
word2vec_model = gensim.downloader.load("glove-wiki-gigaword-50")

# Загрузка токенизатора и модели seq2seq
tokenizer = AutoTokenizer.from_pretrained("t5-small")  # Замените на желаемую модель seq2seq
my_model = AutoModelForSeq2SeqLM.from_pretrained("t5-small")  # Замените на желаемую модель seq2seq

#tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-base")
#model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-base")

# Загрузка стоп-слов
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

[==================================================] 100.0% 66.0/66.0MB downloaded


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Функция расчета PPL(Perplexity). Данная метрика измеряет насколько хорошо модель справляется с предсказанием последовательности слов в тексте.

In [5]:
def calculate_ppl(text, model):
    tokens = word_tokenize(text)
    n = len(tokens)
    ngram = list(nltk.ngrams(tokens, 2))
    ppl = 1.0
    for word1, word2 in ngram:
        # Рассчитываем косинусное расстояние между векторами слов
        try:
            sim = model.similarity(word1, word2)
            # Чем выше сходство (similarity), тем ниже PPL
            ppl *= 1 / (sim + 1e-10)  # Добавляем небольшое значение, чтобы избежать деления на ноль
        except KeyError:
            # Если слово отсутствует в модели, игнорируем его
            pass
    #ppl = min(0.01, ppl)  # Устанавливаем минимальное значение PPL
    #ppl = max(1.0, ppl)  # Устанавливаем максимальное значение PPL
    return ppl

Функция атаки Seq2seq

In [6]:
def attack_sequence_to_sequence_model(input_text, model, max_attempts=10, max_edits=5, with_color=False):
    original_text = input_text
    original_ppl = calculate_ppl(original_text, word2vec_model)  # PPL для оригинального текста
    # Атакуем исходный текст заданное количество раз (максимум max_attempts).
    for _ in range(max_attempts):

        input_ids = tokenizer.encode(input_text, return_tensors="pt", max_length=128, truncation=True, padding=True)
        # Генерируем случайное число изменений (замен) для текущей итерации атаки
        num_edits = random.randint(1, max_edits)
        # Цикл по замене слов в тексте
        for _ in range(num_edits):
            input_word = random.choice(input_ids[0].tolist())
            input_word = tokenizer.convert_ids_to_tokens([input_word])[0]
            # Проверяем, что выбранное слово не является стоп-словом и присутствует в модели Word2Vec
            if input_word not in stop_words and input_word in word2vec_model:
                #  Находим наиболее схоже слова с выбранным словом из модели Word2Vec
                similar_words = word2vec_model.most_similar(input_word, topn=1000)
                # Выбираем одно из схожих слов случайным образом в качестве нового слова
                new_word = random.choice(similar_words)[0]
                if with_color:
                    input_text = input_text.replace(input_word, colored(new_word, 'red'), 1)  # Здесь мы окрашиваем новое слово в красный цвет
                else:
                    input_text = input_text.replace(input_word, new_word, 1)

        # Генерируем новый текст с моделью seq2seq
        generated_ids = my_model.generate(input_ids, max_length=128, num_return_sequences=1)
        generated_text = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
        # вывод оригинального и сгенерированного текста на каждой итерации
        #print(original_text)
        #print(generated_text)
        if generated_text != original_text:
            attacked_ppl = calculate_ppl(generated_text, word2vec_model)  # PPL для атакованного текста

            # Проверка на улучшение PPL после атаки
            if attacked_ppl < original_ppl:

                return generated_text  # Возвращаем атакованный текст
    return None  # Возвращаем None, если атака не удалась




Функция тестирования атаки

In [7]:
def test_attack_and_evaluate(input_text, model, num_attempts=20, max_edits=6):
    original_text = input_text
    print("Оригинальный текст:", original_text)

    attacked_text = attack_sequence_to_sequence_model(original_text, model, max_attempts=num_attempts, max_edits=max_edits)
    if attacked_text:
        print("Атакованный текст:", attacked_text)

        # Оценка PPL
        original_ppl = calculate_ppl(original_text, word2vec_model)
        attacked_ppl = calculate_ppl(attacked_text, word2vec_model)
        print("Original PPL:", original_ppl)
        print("Attacked PPL:", attacked_ppl)

        # Оценка с использованием zlib
        original_compressed = len(zlib.compress(original_text.encode('utf-8')))
        attacked_compressed = len(zlib.compress(attacked_text.encode('utf-8')))
        print("Сжатие оригинального текста:", original_compressed)
        print("Сжатие атакованного текста:", attacked_compressed)

    else:
        print("Атака не удалась.")

# Пример использования функции для тестирования атаки
original_text = "president boris yeltsin stayed home tuesday , nursing a respiratory infection that forced him to cut short a foreign trip and revived concerns about his ability to govern"


test_attack_and_evaluate(original_text, my_model)

Оригинальный текст: president boris yeltsin stayed home tuesday , nursing a respiratory infection that forced him to cut short a foreign trip and revived concerns about his ability to govern
Атакованный текст: boris yeltsin restroth home tuesenter planning nursing a respiratory infection that forced him to cut short a foreign trip and revived concerns about his ability to govern.
Original PPL: 2548072.8625040725
Attacked PPL: 554119.8780239137
Сжатие оригинального текста: 121
Сжатие атакованного текста: 122
